In [ ]:
"""
Chapter 10. Network Models. Max Flow.
Anderson, David R., Dennis J. Sweeney,
Thomas A. Williams,
Jeffrey D. Camm y Kipp Martin
Quantitative Methods for Business, 11a. Ed
"""
from itertools import product
import math

import pulp as pl


In [ ]:

nodes = range(7)

# adjacency_matrix
a = [
    [0, 1, 1, 1, 0, 0, 0],
    [0, 0, 1, 0, 1, 0, 0],
    [0, 1, 0, 1, 1, 1, 0],
    [0, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 1, 1],
    [0, 0, 0, 0, 1, 0, 1],
    [1, 0, 0, 0, 0, 0, 0],
 ]

capacity = [
    [0, 5, 6, 5, 0, 0, 0],
    [0, 0, 2, 0, 3, 0, 0],
    [0, 2, 0, 3, 3, 7, 0],
    [0, 0, 0, 0, 0, 5, 0],
    [0, 0, 0, 0, 0, 1, 8],
    [0, 0, 0, 0, 1, 0, 7],
    [1000, 0, 0, 0, 0, 0, 0],
    ]

load = {
    0: 0, 
    1: 0,
    2: 0,
    3: 0,
    4: 0,
    5: 0,
    6: 0,
}

In [ ]:

capacity = pl.makeDict((nodes, nodes), capacity, 1_000)


# problem
problem = pl.LpProblem('flowmax', pl.LpMaximize)


# variables
send = pl.LpVariable.dicts('send', (nodes, nodes), 0, None, pl.LpContinuous)

# objective function
problem += send[6][0], 'max_flow_network'


# Transhipment Constraints
for k in nodes:
    problem += pl.lpSum(a[k][out] * send[k][out] for out in nodes) - pl.lpSum(a[enter][k] * send[enter][k] for enter in nodes) == load[k], f'send_to_node_{k}'

# capacity constraints
for i in nodes:
    for j in nodes:
        problem += (send[i][j] <= capacity[i][j])

In [ ]:

solver = pl.get_solver('GUROBI') 
problem.solve(solver)

# The status of the solution is printed to the screen
print("Status:", pl.LpStatus[problem.status])


# Each of the variables is printed with it's resolved optimum value
for v in problem.variables():
    if v.varValue > 1e-10:
        print(f'{v.name}: {v.varValue:,.2f}')

print(f'Total Miles = {pl.value(problem.objective):,.2f}')
